In [1]:
import sys
import os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.utils import to_categorical

C:\Users\91956\AppData\Local\Temp\ipykernel_3272\469391663.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Read CSV file
df = pd.read_csv('fer2013.csv')

In [3]:
# Extract and preprocess data
X_train, train_y, X_test, test_y = [], [], [], [] 

In [4]:
for index, row in df.iterrows():
    val = row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val, 'float32'))
            train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val, 'float32'))
            test_y.append(row['emotion'])
    except:
        print(f"error occurred at index: {index} and row: {row}")

In [5]:
# Convert to NumPy arrays
X_train = np.array(X_train, 'float32')
train_y = np.array(train_y, 'float32')
X_test = np.array(X_test, 'float32')
test_y = np.array(test_y, 'float32')


In [6]:
# Convert labels to one-hot encoding
train_y = to_categorical(train_y, num_classes=7)
test_y = to_categorical(test_y, num_classes=7)


In [7]:
# Normalize data between 0 and 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

In [8]:
X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)


In [9]:
# Normalize labels as well
train_y -= np.mean(train_y, axis=0)
train_y /= np.std(train_y, axis=0)

test_y -= np.mean(test_y, axis=0)
test_y /= np.std(test_y, axis=0)


In [10]:
# Reshape data for CNN
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)


In [11]:
# Designing the CNN model
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))


In [12]:
# Compile the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])


In [13]:
# Train the model
model.fit(X_train, train_y,
          batch_size=64,
          epochs=200,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/200


330/449 [=====================>........] - ETA: 43s - loss: -90946619687567360.0000 - accuracy: 0.1429

KeyboardInterrupt: 

In [14]:
# Save the model
fer_json = model.to_json()
with open("fer_2023.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer_2023.h5")

In [15]:
import cv2
import numpy as np
from keras.models import model_from_json

# Load the pre-trained emotion detection model
with open("fer_2023.json", "r") as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("fer_2023.h5")

# Create a list of emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Open a connection to the camera (0 represents the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Preprocess the frame for emotion detection (resize, convert to grayscale, normalize)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = gray[y:y + h, x:x + w]
        resized_face = cv2.resize(face_roi, (48, 48))
        normalized_face = resized_face / 255.0
        reshaped_face = np.reshape(normalized_face, (1, 48, 48, 1))
        emotion_prediction = loaded_model.predict(reshaped_face)
    
        # Get the predicted emotion label
        predicted_emotion = emotion_labels[np.argmax(emotion_prediction)]

        # Draw bounding box and emotion label on the frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Emotion Detection", frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 44ms/step
